In [ ]:
!pip install ipython-autotime
# !pip install -U scikit-learn
%load_ext autotime

time: 1.41 ms (started: 2021-01-23 12:50:06 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.23 ms (started: 2021-01-23 12:50:06 +00:00)


In [ ]:
!pwd
import os
os.chdir('/content/drive/My Drive/ML/ml-project-master/src/')
!pwd

/content
/content/drive/My Drive/ML/ml-project-master/src
time: 237 ms (started: 2021-01-23 12:50:07 +00:00)


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

from utility import *

time: 2.16 s (started: 2021-01-23 12:50:07 +00:00)


In [ ]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

time: 16.7 ms (started: 2021-01-23 12:50:09 +00:00)


# Import Dataset

In [ ]:
def load_cup():
    ml_cup = np.delete(np.genfromtxt('ML-CUP20-TR.csv', 
                                     delimiter=','), obj=0, axis=1)
    return ml_cup[:, :-2], ml_cup[:, -2:]

def load_cup_blind():
    return np.delete(np.genfromtxt('ML-CUP20-TS.csv',
                                   delimiter=','), obj=0, axis=1)
    
def mean_euclidean_error(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    return np.mean(np.linalg.norm(y_pred - y_true, axis=1))

time: 5.79 ms (started: 2021-01-23 12:50:09 +00:00)


In [ ]:
X, y = load_cup()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

time: 27 ms (started: 2021-01-23 12:50:09 +00:00)


In [ ]:
print(X.shape)
print(y.shape)

(1524, 10)
(1524, 2)
time: 1.83 ms (started: 2021-01-23 12:50:09 +00:00)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import L2

# Function to create model, required for KerasRegressor
def create_model(learn_rate=0.01, momentum=0.9, neurons_layer_1=130,
                 init_mode='uniform', activation='sigmoid',  weight_decay = 0.00001,
                 nesterov = True, neurons_layer_2 = 30):
  # create model
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(10)))
    model.add(Dense(neurons_layer_1, activation=activation,
                    kernel_initializer=init_mode, kernel_regularizer= L2(weight_decay)))
    model.add(Dense(neurons_layer_2, activation=activation,
                    kernel_initializer=init_mode, kernel_regularizer= L2(weight_decay)))
    model.add(Dense(2))
    optimizer = SGD(
      learning_rate=learn_rate, momentum=momentum, nesterov=nesterov)
    # Compile model
    model.compile(optimizer=optimizer,
              loss=euclidean_distance_loss)
    return model

time: 14.6 ms (started: 2021-01-23 12:50:09 +00:00)


# Test 1 - Large Grid

In [ ]:
evaluation = []
for i in range(0,3):
  model = create_model(weight_decay= 0.0001)
  history = model.fit(X_train, y_train,
                      batch_size=128,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation.append(error)

np.mean(evaluation)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
10/10 - 0s - loss: 2.5191
Epoch 502/1000
10/10 - 0s - loss: 2.5477
Epoch 503/1000
10/10 - 0s - loss: 2.5177
Epoch 504/1000
10/10 - 0s - loss: 2.5371
Epoch 505/1000
10/10 - 0s - loss: 2.5243
Epoch 506/1000
10/10 - 0s - loss: 2.5375
Epoch 507/1000
10/10 - 0s - loss: 2.5437
Epoch 508/1000
10/10 - 0s - loss: 2.5291
Epoch 509/1000
10/10 - 0s - loss: 2.5173
Epoch 510/1000
10/10 - 0s - loss: 2.5350
Epoch 511/1000
10/10 - 0s - loss: 2.5168
Epoch 512/1000
10/10 - 0s - loss: 2.5243
Epoch 513/1000
10/10 - 0s - loss: 2.5259
Epoch 514/1000
10/10 - 0s - loss: 2.5281
Epoch 515/1000
10/10 - 0s - loss: 2.5125
Epoch 516/1000
10/10 - 0s - loss: 2.5121
Epoch 517/1000
10/10 - 0s - loss: 2.5241
Epoch 518/1000
10/10 - 0s - loss: 2.5275
Epoch 519/1000
10/10 - 0s - loss: 2.5141
Epoch 520/1000
10/10 - 0s - loss: 2.5409
Epoch 521/1000
10/10 - 0s - loss: 2.5269
Epoch 522/1000
10/10 - 0s - loss: 2.5303
Epoch 523/1000
10/10 - 0s - loss: 2.5339
Epoch 5

2.9622945267287744

time: 1min (started: 2021-01-23 12:18:31 +00:00)


In [ ]:
evaluation

[2.94812447148161, 2.9933459318488023, 2.94541317685591]

time: 2.28 ms (started: 2021-01-23 12:20:03 +00:00)


# Test 2 - Large Grid

In [ ]:
evaluation_large_2 = []
for i in range(0,3):
  model = create_model(weight_decay= 0.0001, momentum= 0.8)
  history = model.fit(X_train, y_train,
                      batch_size=64,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation_large_2.append(error)

np.mean(evaluation_large_2)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
20/20 - 0s - loss: 2.6051
Epoch 502/1000
20/20 - 0s - loss: 2.5647
Epoch 503/1000
20/20 - 0s - loss: 2.5699
Epoch 504/1000
20/20 - 0s - loss: 2.5847
Epoch 505/1000
20/20 - 0s - loss: 2.5708
Epoch 506/1000
20/20 - 0s - loss: 2.5757
Epoch 507/1000
20/20 - 0s - loss: 2.5905
Epoch 508/1000
20/20 - 0s - loss: 2.5966
Epoch 509/1000
20/20 - 0s - loss: 2.5810
Epoch 510/1000
20/20 - 0s - loss: 2.5815
Epoch 511/1000
20/20 - 0s - loss: 2.5678
Epoch 512/1000
20/20 - 0s - loss: 2.6010
Epoch 513/1000
20/20 - 0s - loss: 2.5774
Epoch 514/1000
20/20 - 0s - loss: 2.5942
Epoch 515/1000
20/20 - 0s - loss: 2.5948
Epoch 516/1000
20/20 - 0s - loss: 2.5776
Epoch 517/1000
20/20 - 0s - loss: 2.5756
Epoch 518/1000
20/20 - 0s - loss: 2.5707
Epoch 519/1000
20/20 - 0s - loss: 2.5810
Epoch 520/1000
20/20 - 0s - loss: 2.5576
Epoch 521/1000
20/20 - 0s - loss: 2.5650
Epoch 522/1000
20/20 - 0s - loss: 2.6608
Epoch 523/1000
20/20 - 0s - loss: 2.5885
Epoch 5

2.9772544704903736

time: 1min 23s (started: 2021-01-23 12:23:15 +00:00)


In [ ]:
evaluation_large_2

[2.999562789411262, 2.979495436585294, 2.952705185474563]

time: 2.47 ms (started: 2021-01-23 12:24:38 +00:00)


# Test 3 - Large Grid

In [ ]:
evaluation_large_3 = []
for i in range(0,3):
  model = create_model(neurons_layer_1= 70, weight_decay= 0.0001, momentum= 0.8)
  history = model.fit(X_train, y_train,
                      batch_size=128,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation_large_3.append(error)

np.mean(evaluation_large_3)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
10/10 - 0s - loss: 2.6608
Epoch 502/1000
10/10 - 0s - loss: 2.6709
Epoch 503/1000
10/10 - 0s - loss: 2.6677
Epoch 504/1000
10/10 - 0s - loss: 2.6748
Epoch 505/1000
10/10 - 0s - loss: 2.6692
Epoch 506/1000
10/10 - 0s - loss: 2.6616
Epoch 507/1000
10/10 - 0s - loss: 2.6767
Epoch 508/1000
10/10 - 0s - loss: 2.6675
Epoch 509/1000
10/10 - 0s - loss: 2.6540
Epoch 510/1000
10/10 - 0s - loss: 2.6656
Epoch 511/1000
10/10 - 0s - loss: 2.6654
Epoch 512/1000
10/10 - 0s - loss: 2.6661
Epoch 513/1000
10/10 - 0s - loss: 2.6621
Epoch 514/1000
10/10 - 0s - loss: 2.6618
Epoch 515/1000
10/10 - 0s - loss: 2.6621
Epoch 516/1000
10/10 - 0s - loss: 2.6676
Epoch 517/1000
10/10 - 0s - loss: 2.6582
Epoch 518/1000
10/10 - 0s - loss: 2.6599
Epoch 519/1000
10/10 - 0s - loss: 2.6684
Epoch 520/1000
10/10 - 0s - loss: 2.6523
Epoch 521/1000
10/10 - 0s - loss: 2.6664
Epoch 522/1000
10/10 - 0s - loss: 2.6675
Epoch 523/1000
10/10 - 0s - loss: 2.6562
Epoch 5

3.0452045852324297

time: 55.2 s (started: 2021-01-23 12:28:24 +00:00)


In [ ]:
evaluation_large_3

[3.036171903250948, 3.0586908403059794, 3.0407510121403614]

time: 3.14 ms (started: 2021-01-23 12:29:19 +00:00)


# Test 4 - Large Grid

In [ ]:
evaluation_large_4 = []
for i in range(0,3):
  model = create_model(weight_decay= 0.0001, momentum= 0.8)
  history = model.fit(X_train, y_train,
                      batch_size=128,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation_large_4.append(error)

np.mean(evaluation_large_4)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
10/10 - 0s - loss: 2.7356
Epoch 502/1000
10/10 - 0s - loss: 2.7300
Epoch 503/1000
10/10 - 0s - loss: 2.7344
Epoch 504/1000
10/10 - 0s - loss: 2.7254
Epoch 505/1000
10/10 - 0s - loss: 2.7315
Epoch 506/1000
10/10 - 0s - loss: 2.7264
Epoch 507/1000
10/10 - 0s - loss: 2.7299
Epoch 508/1000
10/10 - 0s - loss: 2.7339
Epoch 509/1000
10/10 - 0s - loss: 2.7332
Epoch 510/1000
10/10 - 0s - loss: 2.7318
Epoch 511/1000
10/10 - 0s - loss: 2.7271
Epoch 512/1000
10/10 - 0s - loss: 2.7259
Epoch 513/1000
10/10 - 0s - loss: 2.7287
Epoch 514/1000
10/10 - 0s - loss: 2.7267
Epoch 515/1000
10/10 - 0s - loss: 2.7259
Epoch 516/1000
10/10 - 0s - loss: 2.7235
Epoch 517/1000
10/10 - 0s - loss: 2.7229
Epoch 518/1000
10/10 - 0s - loss: 2.7229
Epoch 519/1000
10/10 - 0s - loss: 2.7378
Epoch 520/1000
10/10 - 0s - loss: 2.7227
Epoch 521/1000
10/10 - 0s - loss: 2.7187
Epoch 522/1000
10/10 - 0s - loss: 2.7239
Epoch 523/1000
10/10 - 0s - loss: 2.7219
Epoch 5

3.0108705102477775

time: 57.6 s (started: 2021-01-23 12:31:55 +00:00)


In [ ]:
evaluation_large_4

[2.998907711459215, 3.013216063323153, 3.0204877559609655]

time: 2.81 ms (started: 2021-01-23 12:32:53 +00:00)


# Test 5 - Large Grid

In [ ]:
evaluation_large_5 = []
for i in range(0,3):
  model = create_model(neurons_layer_1= 70, weight_decay= 0.0001, momentum= 0.9, learn_rate=0.001, activation='tanh')
  history = model.fit(X_train, y_train,
                      batch_size=128,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation_large_5.append(error)

np.mean(evaluation_large_5)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
10/10 - 0s - loss: 2.6796
Epoch 502/1000
10/10 - 0s - loss: 2.6645
Epoch 503/1000
10/10 - 0s - loss: 2.6632
Epoch 504/1000
10/10 - 0s - loss: 2.6643
Epoch 505/1000
10/10 - 0s - loss: 2.6615
Epoch 506/1000
10/10 - 0s - loss: 2.6624
Epoch 507/1000
10/10 - 0s - loss: 2.6642
Epoch 508/1000
10/10 - 0s - loss: 2.6636
Epoch 509/1000
10/10 - 0s - loss: 2.6624
Epoch 510/1000
10/10 - 0s - loss: 2.6620
Epoch 511/1000
10/10 - 0s - loss: 2.6569
Epoch 512/1000
10/10 - 0s - loss: 2.6589
Epoch 513/1000
10/10 - 0s - loss: 2.6572
Epoch 514/1000
10/10 - 0s - loss: 2.6579
Epoch 515/1000
10/10 - 0s - loss: 2.6548
Epoch 516/1000
10/10 - 0s - loss: 2.6568
Epoch 517/1000
10/10 - 0s - loss: 2.6572
Epoch 518/1000
10/10 - 0s - loss: 2.6538
Epoch 519/1000
10/10 - 0s - loss: 2.6595
Epoch 520/1000
10/10 - 0s - loss: 2.6536
Epoch 521/1000
10/10 - 0s - loss: 2.6530
Epoch 522/1000
10/10 - 0s - loss: 2.6504
Epoch 523/1000
10/10 - 0s - loss: 2.6563
Epoch 5

2.938435408576821

time: 55.4 s (started: 2021-01-23 12:50:16 +00:00)


In [ ]:
evaluation_large_5

[2.9489056802516234, 2.963252981534383, 2.903147563944458]

time: 3.31 ms (started: 2021-01-23 12:51:11 +00:00)


# Test 1 - Granular Grid

In [ ]:
evaluation_granular_1 = []
for i in range(0,3):
  model = create_model(neurons_layer_1= 130, weight_decay= 0.00001, momentum= 0.9)
  history = model.fit(X_train, y_train,
                      batch_size=128,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation_granular_1.append(error)

np.mean(evaluation_granular_1)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
10/10 - 0s - loss: 2.4946
Epoch 502/1000
10/10 - 0s - loss: 2.4870
Epoch 503/1000
10/10 - 0s - loss: 2.4977
Epoch 504/1000
10/10 - 0s - loss: 2.4870
Epoch 505/1000
10/10 - 0s - loss: 2.4704
Epoch 506/1000
10/10 - 0s - loss: 2.4721
Epoch 507/1000
10/10 - 0s - loss: 2.4774
Epoch 508/1000
10/10 - 0s - loss: 2.4797
Epoch 509/1000
10/10 - 0s - loss: 2.4638
Epoch 510/1000
10/10 - 0s - loss: 2.4731
Epoch 511/1000
10/10 - 0s - loss: 2.4685
Epoch 512/1000
10/10 - 0s - loss: 2.4874
Epoch 513/1000
10/10 - 0s - loss: 2.4765
Epoch 514/1000
10/10 - 0s - loss: 2.4721
Epoch 515/1000
10/10 - 0s - loss: 2.4878
Epoch 516/1000
10/10 - 0s - loss: 2.4614
Epoch 517/1000
10/10 - 0s - loss: 2.4614
Epoch 518/1000
10/10 - 0s - loss: 2.4924
Epoch 519/1000
10/10 - 0s - loss: 2.4900
Epoch 520/1000
10/10 - 0s - loss: 2.4743
Epoch 521/1000
10/10 - 0s - loss: 2.4775
Epoch 522/1000
10/10 - 0s - loss: 2.4752
Epoch 523/1000
10/10 - 0s - loss: 2.4622
Epoch 5

2.968224646138424

time: 59.4 s (started: 2021-01-23 12:46:23 +00:00)


In [ ]:
evaluation_granular_1

[2.978514260253916, 2.9546816142001844, 2.971478063961172]

time: 2.37 ms (started: 2021-01-23 12:47:22 +00:00)


# Test 2 - Granular Grid

In [ ]:
evaluation_granular_2 = []
for i in range(0,3):
  model = create_model(neurons_layer_1= 130, weight_decay= 0.00001, momentum= 0.8)
  history = model.fit(X_train, y_train,
                      batch_size=64,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation_granular_2.append(error)

np.mean(evaluation_granular_2)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
20/20 - 0s - loss: 2.5753
Epoch 502/1000
20/20 - 0s - loss: 2.6021
Epoch 503/1000
20/20 - 0s - loss: 2.5971
Epoch 504/1000
20/20 - 0s - loss: 2.5768
Epoch 505/1000
20/20 - 0s - loss: 2.5799
Epoch 506/1000
20/20 - 0s - loss: 2.6363
Epoch 507/1000
20/20 - 0s - loss: 2.5869
Epoch 508/1000
20/20 - 0s - loss: 2.5812
Epoch 509/1000
20/20 - 0s - loss: 2.5899
Epoch 510/1000
20/20 - 0s - loss: 2.5855
Epoch 511/1000
20/20 - 0s - loss: 2.5820
Epoch 512/1000
20/20 - 0s - loss: 2.5921
Epoch 513/1000
20/20 - 0s - loss: 2.5709
Epoch 514/1000
20/20 - 0s - loss: 2.6114
Epoch 515/1000
20/20 - 0s - loss: 2.6417
Epoch 516/1000
20/20 - 0s - loss: 2.6142
Epoch 517/1000
20/20 - 0s - loss: 2.6011
Epoch 518/1000
20/20 - 0s - loss: 2.6087
Epoch 519/1000
20/20 - 0s - loss: 2.5739
Epoch 520/1000
20/20 - 0s - loss: 2.5592
Epoch 521/1000
20/20 - 0s - loss: 2.5721
Epoch 522/1000
20/20 - 0s - loss: 2.5650
Epoch 523/1000
20/20 - 0s - loss: 2.5737
Epoch 5

2.9712275939884876

time: 1min 23s (started: 2021-01-23 12:51:14 +00:00)


In [ ]:
evaluation_granular_2

[2.978136097432234, 2.965908017201986, 2.9696386673312425]

time: 2.59 ms (started: 2021-01-23 12:52:38 +00:00)


# Test 3 - Granular Grid

In [ ]:
evaluation_granular_3 = []
for i in range(0,3):
  model = create_model(neurons_layer_1= 130, weight_decay= 0.00001, momentum= 0.7)
  history = model.fit(X_train, y_train,
                      batch_size=64,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation_granular_3.append(error)

np.mean(evaluation_granular_3)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
20/20 - 0s - loss: 2.6685
Epoch 502/1000
20/20 - 0s - loss: 2.6617
Epoch 503/1000
20/20 - 0s - loss: 2.6842
Epoch 504/1000
20/20 - 0s - loss: 2.6964
Epoch 505/1000
20/20 - 0s - loss: 2.6761
Epoch 506/1000
20/20 - 0s - loss: 2.6765
Epoch 507/1000
20/20 - 0s - loss: 2.6607
Epoch 508/1000
20/20 - 0s - loss: 2.6700
Epoch 509/1000
20/20 - 0s - loss: 2.6773
Epoch 510/1000
20/20 - 0s - loss: 2.7160
Epoch 511/1000
20/20 - 0s - loss: 2.6915
Epoch 512/1000
20/20 - 0s - loss: 2.6919
Epoch 513/1000
20/20 - 0s - loss: 2.7171
Epoch 514/1000
20/20 - 0s - loss: 2.6585
Epoch 515/1000
20/20 - 0s - loss: 2.6600
Epoch 516/1000
20/20 - 0s - loss: 2.6614
Epoch 517/1000
20/20 - 0s - loss: 2.6662
Epoch 518/1000
20/20 - 0s - loss: 2.6867
Epoch 519/1000
20/20 - 0s - loss: 2.6507
Epoch 520/1000
20/20 - 0s - loss: 2.6538
Epoch 521/1000
20/20 - 0s - loss: 2.6838
Epoch 522/1000
20/20 - 0s - loss: 2.6595
Epoch 523/1000
20/20 - 0s - loss: 2.6609
Epoch 5

3.0423544286393245

time: 1min 24s (started: 2021-01-23 12:55:50 +00:00)


In [ ]:
evaluation_granular_3

[2.9758902602349027, 2.9351630404887263, 3.216009985194345]

time: 8.71 ms (started: 2021-01-23 12:57:14 +00:00)


# Test 4 - Granular Grid

In [ ]:
evaluation_granular_4 = []
for i in range(0,3):
  model = create_model(neurons_layer_1= 130, weight_decay= 0.00001, momentum= 0.8)
  history = model.fit(X_train, y_train,
                      batch_size=128,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation_granular_4.append(error)

np.mean(evaluation_granular_4)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
10/10 - 0s - loss: 2.7569
Epoch 502/1000
10/10 - 0s - loss: 2.7648
Epoch 503/1000
10/10 - 0s - loss: 2.7610
Epoch 504/1000
10/10 - 0s - loss: 2.7571
Epoch 505/1000
10/10 - 0s - loss: 2.7822
Epoch 506/1000
10/10 - 0s - loss: 2.7563
Epoch 507/1000
10/10 - 0s - loss: 2.7688
Epoch 508/1000
10/10 - 0s - loss: 2.7818
Epoch 509/1000
10/10 - 0s - loss: 2.7647
Epoch 510/1000
10/10 - 0s - loss: 2.7612
Epoch 511/1000
10/10 - 0s - loss: 2.7566
Epoch 512/1000
10/10 - 0s - loss: 2.7593
Epoch 513/1000
10/10 - 0s - loss: 2.7527
Epoch 514/1000
10/10 - 0s - loss: 2.7556
Epoch 515/1000
10/10 - 0s - loss: 2.7519
Epoch 516/1000
10/10 - 0s - loss: 2.7466
Epoch 517/1000
10/10 - 0s - loss: 2.7487
Epoch 518/1000
10/10 - 0s - loss: 2.7592
Epoch 519/1000
10/10 - 0s - loss: 2.7423
Epoch 520/1000
10/10 - 0s - loss: 2.7443
Epoch 521/1000
10/10 - 0s - loss: 2.7481
Epoch 522/1000
10/10 - 0s - loss: 2.7514
Epoch 523/1000
10/10 - 0s - loss: 2.7433
Epoch 5

2.9271303851161528

time: 1min (started: 2021-01-23 12:59:19 +00:00)


In [ ]:
evaluation_granular_4

[2.921164627348345, 2.914560626443049, 2.9456659015570645]

time: 2.95 ms (started: 2021-01-23 13:00:20 +00:00)


# Test 5 - Granular Grid

In [ ]:
evaluation_granular_5 = []
for i in range(0,3):
  model = create_model(neurons_layer_1= 70, weight_decay= 0.00001, momentum= 0.7, learn_rate= 0.01)
  history = model.fit(X_train, y_train,
                      batch_size=128,
                      epochs= 1000,
                      verbose=2)
  error = mean_euclidean_error(model.predict(X_test), y_test)
  evaluation_granular_5.append(error)

np.mean(evaluation_granular_5)

Output streaming troncato alle ultime 5000 righe.
Epoch 501/1000
10/10 - 0s - loss: 2.8567
Epoch 502/1000
10/10 - 0s - loss: 2.8552
Epoch 503/1000
10/10 - 0s - loss: 2.8512
Epoch 504/1000
10/10 - 0s - loss: 2.8644
Epoch 505/1000
10/10 - 0s - loss: 2.8524
Epoch 506/1000
10/10 - 0s - loss: 2.8514
Epoch 507/1000
10/10 - 0s - loss: 2.8496
Epoch 508/1000
10/10 - 0s - loss: 2.8488
Epoch 509/1000
10/10 - 0s - loss: 2.8488
Epoch 510/1000
10/10 - 0s - loss: 2.8421
Epoch 511/1000
10/10 - 0s - loss: 2.8415
Epoch 512/1000
10/10 - 0s - loss: 2.8498
Epoch 513/1000
10/10 - 0s - loss: 2.8509
Epoch 514/1000
10/10 - 0s - loss: 2.8480
Epoch 515/1000
10/10 - 0s - loss: 2.8498
Epoch 516/1000
10/10 - 0s - loss: 2.8413
Epoch 517/1000
10/10 - 0s - loss: 2.8450
Epoch 518/1000
10/10 - 0s - loss: 2.8385
Epoch 519/1000
10/10 - 0s - loss: 2.8434
Epoch 520/1000
10/10 - 0s - loss: 2.8327
Epoch 521/1000
10/10 - 0s - loss: 2.8433
Epoch 522/1000
10/10 - 0s - loss: 2.8358
Epoch 523/1000
10/10 - 0s - loss: 2.8443
Epoch 5

2.935782299423766

time: 55.9 s (started: 2021-01-23 13:07:46 +00:00)


In [ ]:
evaluation_granular_5

[2.9066724547062255, 2.941576051721992, 2.9590983918430815]

time: 3.57 ms (started: 2021-01-23 13:08:42 +00:00)
